# 1.1 Facebook Comments Scraper
- Please update the following FINAL variables to indicate root data folder and user credentials
- error_log.txt file needs to be created before hand within the root data folder
- Additionally, please follow the folder structure below

```
FacebookScraper/
|-- Data/
|   |-- DollarsAndSense/
|   |-- MortgageConsultancy/
|   |-- Seedly/
|   |-- WokeManSalary/
|   |   |-- comments/
|   |   |   |-- WokeManSalary_comments_[xxx].csv
|   |   |   |-- ...
|   |   |   |-- ...
|   |   |-- WokeManSalary_posts.csv
|   |-- error_log.txt
|-- Facebook_Comments.ipynb
```

In [1]:
# Declaration of FINAL Static variables
ROOT_DATA_FOLDER = './Data/' 
EMAIL = 'vintagemustang_point@hotmail.com' # <--- Change to own account when using
PASSWORD = 'micheal_tan_300' # <--- Change to own password when using

# Declaration of Dynamic variables
page_list = [
    'DollarsAndSense',
    'MortgageConsultancy',
    'Seedly',
    'WokeManSalary'
]

In [2]:
from selenium import webdriver # allow launching browser
from selenium.webdriver.support.ui import WebDriverWait # allow waiting for page to load
from bs4 import BeautifulSoup # to parse HTML objects

import os
import csv
import time
import traceback
import datetime

In [3]:
driver_option = webdriver.ChromeOptions()
driver_option.add_argument('--headless')

# driver_option.add_argument('--incognito')
chromedriver_path = '/Users/alfaried/Google Drive/SMU - Y04S01/IS434 SA/Project/chromedriver' # <--- Change this to your own chromedriver path!

def create_webdriver():
    return webdriver.Chrome(executable_path=chromedriver_path, chrome_options=driver_option)

def login(driver, url):
    
    # Open the website
    driver.implicitly_wait(5)
    driver.get('https://mbasic.facebook.com' + url)

    # Clicks on "Log In" button
    root_element = driver.find_element_by_id('root')
    login_element = root_element.find_element_by_tag_name('a').click()
    time.sleep(1)

    # Input username
    username_input = driver.find_element_by_name("email")
    username_input.clear()
    username_input.send_keys(EMAIL)

    # Input password
    password_input = driver.find_element_by_name("pass")
    password_input.clear()
    password_input.send_keys(PASSWORD)

    # Logs into Facebook using dummy account
    driver.find_element_by_name("login").click()
    time.sleep(1)
    
    return driver

def get_comments_post(driver, comment_link):  
    comment_list = []
    
    # Get comments
    element = driver.find_element_by_xpath('//*[@id="ufi_' + comment_link.split('=')[1].split('&')[0] + '"]/div')

    comment_content_div = element.get_attribute('innerHTML')
    soup = BeautifulSoup(comment_content_div, 'html.parser')

    comment_chunk = soup.contents[-2]
    comments = comment_chunk.contents
    
    if comment_chunk.get('id') == None:         
        for comment in comments:
            try:             
                # Get comment ID
                comment_id = comment.get('id')
                
                # Get user and contents of comments
                comment_user = comment.find('h3').get_text()
                comment_text = comment.find_all('div')[1].get_text()
                comment_date = comment.find('abbr').get_text()
                comment_list.append({
                    'user': comment_user,
                    'text': comment_text,
                    'date': comment_date,
                    'id': comment_id
                })  
            except:
                # If there's a "View more comments..." link
                if 'see_next' in comment.get('id'):
                    driver.get('https://mbasic.facebook.com' + comment.find('a')['href'])

                    tmp_list = get_comments_post(driver, comment_link)

                    if tmp_list != []:
                        comment_list = comment_list + tmp_list

    return comment_list

def get_comments_photo_posts(driver, comment_link):
    comment_list = []
    
    # Get comments
    comment_element = driver.find_element_by_xpath('//*[@id="ufi_' + comment_link.split('/')[-1] + '"]')
    
    comment_content = comment_element.get_attribute('innerHTML')
    comment_soup = BeautifulSoup(comment_content, 'html.parser')
    
    comment_chunk = comment_soup.contents[0].contents[-2]
    
    if comment_chunk.get('id') == None:  
        comments = comment_chunk.contents

        for comment in comments:
            try:
                # Get comment ID
                comment_id = comment.get('id')
                
                # Get user and contents of comments
                comment_user = comment.find('h3').get_text()
                comment_text = comment.find_all('div')[1].get_text()
                comment_date = comment.find('abbr').get_text()
                comment_list.append({
                    'user': comment_user,
                    'text': comment_text,
                    'date': comment_date,
                    'id': comment_id
                }) 
            except:
                # If there's a "View more comments..." link
                if 'see_next' in comment.get('id'):
                    driver.get('https://mbasic.facebook.com' + comment.find('a')['href'])

                    tmp_list = get_comments_photo_posts(driver, comment_link)

                    if tmp_list != []:
                        comment_list = comment_list + tmp_list
                        
    return comment_list

def read_urls_csv(filepath):   
    url_list = []
    print('Reading ' + filepath.split('/')[-1] + '...')
    print('')
    
    with open(filepath, mode='r') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if 'url' not in row[-1]:
                url_list.append(row[-1])
            
    return url_list

def write_comments_csv(filepath, comments_data):
    data_size = len(comments_data)
    
    if data_size == 0:
        print('No comments')
        print('')
        return
    
    print(str(data_size) + ' comment(s)')
    print('Writing ' + filepath.split('/')[-1] + '...')
    print('')

    with open(filepath, mode='w') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

        # Write Header
        csv_writer.writerow(['User', 'Comment', 'Date', 'Comment_ID'])

        # Write Content
        for comment in comments_data:
            tmp = comment['date'].split(' ')
            dateTime = '/'.join(tmp[:2] + ['19']) + ' ' + tmp[-1]

            csv_writer.writerow([comment['user'], comment['text'],  dateTime, comment['id']])
            
def write_errors(page, url):
    print('ERROR with URL: ' + url)
    print('Writing to error_log.txt...')
    print('')
    
    with open(ROOT_DATA_FOLDER + 'error_log.txt', mode='a') as csv_file:
        now = datetime.datetime.now()
        
        # Write Content
        csv_file.write('[' + str(now) + '] ERROR for ' + page + '. URL: ' + url + '\n\n')

In [4]:
for page in page_list:
    url_list = read_urls_csv(ROOT_DATA_FOLDER + page + '/' +  page + '_posts.csv')
    comments_folder = ROOT_DATA_FOLDER + page + '/Comments/'
    
    # Loads files in comment folders
    comment_folder_files = os.listdir(comments_folder)

    for url in url_list: 
        post_id = url.split('=')[1].split('&')[0] if 'story' in url else url.split('/')[-1]
        comments_filename = page + '_comments_' + post_id + '.csv'
        
        # Checks if file comments already exists in folder. If TRUE, skips.
        if comments_filename in comment_folder_files:
            print(post_id + ' already scraped')
            print('Skipping...')
            print('')
            continue
        
        # STARTS scraping comments from posts here
        print('Retrieving comments from ' + url)
        comments_data = None  
        comments_filepath = comments_folder + comments_filename
        
        try:
            # Get webdriver Object
            driver = create_webdriver()

            # Log into Facebook
            driver = login(driver, url)
            
            # Retrieve comments
            if 'story' in url:            
                comments_data = get_comments_post(driver, url)            
            else:
                comments_data = get_comments_photo_posts(driver, url)
            
            # Writes to CSV file
            write_comments_csv(comments_filepath, comments_data)
            
        except:
            # Handles exceptions
            write_errors(page, url)
            
            # Uncomment to print stacktrace
            # traceback.print_exc()
                
        finally:
            # Close the Webdriver
            driver.close()
        
    print('Scraping comments for ' + page + ' posts, done')
    print('')

Reading DollarsAndSense_posts.csv...

Retrieving comments from /story.php?story_fbid=3182079765195405&id=386868888049854


/Users/alfaried/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


ERROR with URL: /story.php?story_fbid=3182079765195405&id=386868888049854
Writing to error_log.txt...

3182609391809109 already scraped
Skipping...

3180221452047903 already scraped
Skipping...

3179548878781827 already scraped
Skipping...

3179059135497468 already scraped
Skipping...

Retrieving comments from /story.php?story_fbid=3175929855810396&id=386868888049854
ERROR with URL: /story.php?story_fbid=3175929855810396&id=386868888049854
Writing to error_log.txt...

3172813522788696 already scraped
Skipping...

Retrieving comments from /story.php?story_fbid=3170083759728339&id=386868888049854
ERROR with URL: /story.php?story_fbid=3170083759728339&id=386868888049854
Writing to error_log.txt...

3169627359773979 already scraped
Skipping...

3166908160045899 already scraped
Skipping...

3166688096734572 already scraped
Skipping...

3164494630287252 already scraped
Skipping...

3163912817012100 already scraped
Skipping...

3162719090464806 already scraped
Skipping...

3160511200685595 al

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=55978): Max retries exceeded with url: /session/fe12753d9673ddff1758eb941ff8b0de/window (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10fcf7c88>: Failed to establish a new connection: [Errno 61] Connection refused'))

# 1.2 Cleaning and Collating of Data
- Collating the comments which are categorized by post ids
- Cleaning and standardizing collated comments


In [ ]:
import os
import csv
import pandas
import traceback
import datetime

In [ ]:
def read_comments_csv():
    return

def handle_dates():
    return

def write_data_csv():
    return

In [ ]:
# .. Start Buffer ..

In [ ]:
# ..

In [ ]:
# ..

In [ ]:
# ..

In [ ]:
# ..

In [ ]:
# .. End Buffer ..

# 2.1 Testing Section
Methods from aboved portion can be tested and used here, just as long as they are successfully compiled

In [271]:
# urls_DollarsAndSense = [
#     '/DollarsAndSenseSG/photos/a.1697485630321500/2774662429270476',
#     '/DollarsAndSenseSG/photos/a.900467096690028/2751670081569711',
#     '/DollarsAndSenseSG/photos/a.1697485630321500/2746837705386282',
#     '/DollarsAndSenseSG/photos/a.1697485630321500/2744233652313354',
#     '/DollarsAndSenseSG/photos/a.900467096690028/2694911627245557',
#     '/DollarsAndSenseSG/photos/a.900467096690028/2688094337927286'
# ]

# urls_ConsultMortgage = [
#     '/consultmortgage/photos/a.1455344844620117/1455344894620112'
# ]

# urls_Seedly = [
#     '/seedlysg/photos/a.470073689847337/1059294577591909',
#     '/seedlysg/photos/a.470073689847337/1143479625840070',
#     '/seedlysg/photos/a.470073689847337/1111985378989495'
# ]


# for url in urls_Seedly:
#     comment_list = None
    
#     try:
#         # Get webdriver Object
#         driver = create_webdriver()

#         # Open the website
#         driver.implicitly_wait(5)
#         driver.get('https://mbasic.facebook.com' + url)

#         # Clicks on "Log In" button
#         root_element = driver.find_element_by_id('root')
#         login_element = root_element.find_element_by_tag_name('a').click()
#         time.sleep(1)

#         # Input username
#         username_input = driver.find_element_by_name("email")
#         username_input.clear()
#         username_input.send_keys('vintagemustang_point@hotmail.com')

#         # Input password
#         password_input = driver.find_element_by_name("pass")
#         password_input.clear()
#         password_input.send_keys('micheal_tan_300')

#         # Logs into Facebook using dummy account
#         driver.find_element_by_name("login").click()
#         time.sleep(1)

#         # Retrieve comments from photo posts
#         comment_list = get_comments_photo_posts(driver, url)
        
#         print(url)
#         print(comment_list)

#     except:
#         traceback.print_exc()

#     finally:
#         # Close the Webdriver
#         driver.close()

In [356]:
# url = '/story.php?story_fbid=2921056877964363&id=386868888049854'
# comments_data = None

# try:
#     # Get webdriver Object
#     driver = create_webdriver()

#     # Open the website
#     driver.implicitly_wait(5)
#     driver.get('https://mbasic.facebook.com' + url)
    
#     # Clicks on "Log In" button
#     root_element = driver.find_element_by_id('root')
#     login_element = root_element.find_element_by_tag_name('a').click()
#     time.sleep(1)

#     # tmp = root_element.get_attribute('innerHTML')
#     # print(tmp)
#     # soup = tmp(tmp, 'html.parser')
#     # print(soup)
    
#     # Input username
#     username_input = driver.find_element_by_name("email")
#     username_input.clear()
#     username_input.send_keys('vintagemustang_point@hotmail.com')

#     # Input password
#     password_input = driver.find_element_by_name("pass")
#     password_input.clear()
#     password_input.send_keys('micheal_tan_300')

#     # Logs into Facebook using dummy account
#     driver.find_element_by_name("login").click()
#     time.sleep(1)

#     # Retrieve comments
#     comments_data = get_comments_post(driver, url)
#     print(comments_data)
# except:
#     traceback.print_exc()

# finally:
#     # Close the Webdriver
#     driver.close()

/Users/alfaried/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


<div class="m n"><form method="post" action="https://mbasic.facebook.com/story.php?story_fbid=2921056877964363&amp;id=386868888049854&amp;refid=52"><input type="hidden" name="lsd" value="AVov7IKv" autocomplete="off"><input type="hidden" name="jazoest" value="2701" autocomplete="off"><div id="captcha" class="o"><div>Please enter the code below</div><input type="hidden" id="captcha_persist_data" name="captcha_persist_data" value="AZn50_3OWrCIaGw3R8OB3rUOxtgitehfLqIHHsxk4ZaxagU2WOiBuvW0CkWVe4Rb3PKRuC1WU9ddoupj38sn3qn3kIQfVtVd8ZsWdhFY_LDTDp1EfF_UXrkQbZQbhBaLiRvDx7d4FVC1h-roe-pgswa1hqlj04yrEkRNpiBc1L1mAAUZ7tt5BEG5G_X9HiTPEcRZfiDlB7f_ZIuUTT_umDGCNltyatC0-quapDKzb74BrUVuVZ4wD0AVg0KLsXwTeGVCrwcbTWShe7e-NJNsdLAm-WIOKd1_O12DBtgHECzTXby8zsxFG2GIYYddQexL2P98eJKjvADStyagR16Dp6_r8FOuExTduo_fSppstvu-ltBJHLdWA9siR9LN5YjZycR4LoiSJ6gB9t8Pb3lRMMAxAJQv6XW_UQ53Uj_2q4H5mu86oD-wcV663qzeSAc9H2A"><img src="https://mbasic.facebook.com/captcha/tfbimage.php?captcha_challenge_code=1572158837-72956f31519c5d21213d02

Traceback (most recent call last):
  File "<ipython-input-356-fbeb578df4d7>", line 19, in <module>
    soup = tmp(tmp, 'html.parser')
TypeError: 'str' object is not callable
